In [1]:
class EXP:
    MODEL = 'sentence-transformers/sentence-t5-base'
    # MODEL = 'Alibaba-NLP/gte-base-en-v1.5'
    # MODEL = 'Alibaba-NLP/gte-large-en-v1.5'
    # MODEL = 'intfloat/e5-mistral-7b-instruct'
    # MODEL = 'BAAI/bge-base-en-v1.5'
    # MODEL = "Alibaba-NLP/gte-Qwen2-1.5B-instruct"
    # MODEL = "Alibaba-NLP/gte-Qwen2-7B-instruct"
    # MODEL = "princeton-nlp/sup-simcse-bert-large-uncased"
    # MODEL = "Salesforce/SFR-Embedding-2_R"
    # MODEL = "nomic-ai/nomic-embed-text-v1"
    # MODEL = "hkunlp/instructor-base"
    # MODEL = "hkunlp/instructor-xl"
    # MODEL = "WhereIsAI/UAE-Large-V1"
    # MODEL = "nomic-ai/nomic-bert-2048"
    # MODEL = "thenlper/gte-small"
    # MODEL = "Salesforce/SFR-Embedding-2_R"
    # MODEL = "GritLM/GritLM-7B"
    # MODEL = "sentence-transformers/gtr-t5-base"
    # MODEL = "nomic-ai/nomic-embed-text-v1.5"
    # MODEL = 'sentence-transformers/all-mpnet-base-v2'
    # MODEL  = "sentence-transformers/all-MiniLM-L6-v2"
    DATASET = [
        "mteb/sts13-sts",
        "mteb/sts22-crosslingual-sts",
        "mteb/sts12-sts",
        "mteb/stsbenchmark-sts",
        "mteb/sickr-sts",
        "mteb/sts14-sts",
        "mteb/biosses-sts",
        "mteb/sts16-sts",
        "mteb/sts15-sts",
        "mteb/stsb_multi_mt",
        "mteb/sts17-crosslingual-sts"
    ]
    SENT_PAIR_NUM = 5
    INSERT_NUM = 8
    VERIFICATION_SENT_PAIR_NUM = 250

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
sys.path.append('/root/StickyToken')
# from stickytoken.utils import *

In [3]:
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer(EXP.MODEL                
#                             ,model_kwargs={
#                     "torch_dtype": "bfloat16",
#                     "attn_implementation": "flash_attention_2"})
# model = SentenceTransformer(EXP.MODEL)
# model._first_module().auto_model.embed_tokens.weight.data.detach().cpu().numpy()
# model._first_module().auto_model.embeddings.word_embeddings.weight.data.detach().cpu().numpy()

In [ ]:
# from sentence_transformers import SentenceTransformer

# model = SentenceTransformer("nomic-ai/nomic-embed-text-v1")

In [ ]:
from stickytoken.embedding_model import ModelAnalyzer
moda = ModelAnalyzer(EXP.MODEL)

In [ ]:
moda.vocab_embeddings

In [ ]:
from stickytoken.tokenization import TokenizerAnalyzer
toka = TokenizerAnalyzer(EXP.MODEL)

In [ ]:
token_infos = toka.categorize_tokens()

In [ ]:
moda.check_on_unit_sphere()

In [ ]:
moda.check_is_anisotropic()

In [ ]:
from stickytoken.sentence_pair import SentencePair,Dataset
sent_pairs = SentencePair(EXP.DATASET,moda.model,moda.model_name)

In [ ]:
dataset = Dataset(sent_pairs.sampled_sentence_pairs_path, moda, EXP.SENT_PAIR_NUM)

In [ ]:
verification_dataset = Dataset(sent_pairs.sampled_sentence_pairs_path, moda, EXP.VERIFICATION_SENT_PAIR_NUM)

In [11]:
# moda.neighbor_distances_statistics(mode = 'mean')
# moda.neighbor_distances_statistics()
# moda.cosine_threshold,moda.euclidean_threshold,moda.manhattan_threshold

In [12]:
moda.magic_token_test('</s>',None,dataset,EXP.INSERT_NUM,)
# moda.magic_token_test('</s>',5058,dataset,EXP.INSERT_NUM,)
# moda.magic_results

In [ ]:
token_infos_with_metrics = moda.caculate_vocab_token_magic_score(token_infos,
                                                                 dataset,
                                                                 EXP.INSERT_NUM,
                                                                 do_sample=True,
                                                                 sample_num=500,)
# token_infos_with_metric

In [ ]:
token_infos_with_metrics = moda.caculate_vocab_token_magic_score_multi_token(token_infos,
                                                                 dataset,
                                                                 EXP.INSERT_NUM,
                                                                 do_sample=True,
                                                                 sample_num=500,)
# token_infos_with_metric

In [ ]:
# moda.magic_token_verify('</s>',None,verification_dataset,EXP.INSERT_NUM)
# moda.magic_token_verify('</s>',5633,verification_dataset,EXP.INSERT_NUM)
# moda.magic_token_verify('</s>',19784,verification_dataset,8)
# moda.verify_results

In [ ]:
import numpy as np
from stickytoken.utils import load_vocab_token_magic_scores
token_infos_with_metrics = load_vocab_token_magic_scores(os.path.basename(EXP.MODEL))
import pandas as pd
import pygwalker as pyg
import matplotlib.pyplot as plt
df_token_infos_with_metrics = pd.DataFrame(token_infos_with_metrics).T

# 过滤掉 'main_metric' 列中的 NaN 值
filtered_main_metric = df_token_infos_with_metrics['main_metric'].dropna()

# 绘制箱线图并获取异常点
plt.figure(figsize=(10, 6))
boxplot_result = plt.boxplot(filtered_main_metric)
plt.title('Boxplot of Main Metric')
plt.ylabel('Main Metric')
plt.show()

# 计算四分位数和四分位间距
Q1 = filtered_main_metric.quantile(0.25)
median = filtered_main_metric.median()
Q3 = filtered_main_metric.quantile(0.75)
IQR = Q3 - Q1

# 计算上下边界
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 找出异常值
outliers = filtered_main_metric[(filtered_main_metric < lower_bound) | (filtered_main_metric > upper_bound)]
print("下四分位数 Q1:", Q1)
print("中位数:", median)
print("上四分位数 Q3:", Q3)
print("四分位间距 IQR:", IQR)
print("下边界:", lower_bound)
print("上边界:", upper_bound)
print("异常值:", outliers.values)
print("异常值数量:", len(outliers))

# 找出filter后的main_metric中的前2%的值
threshold = np.percentile(filtered_main_metric, 98)
top_2_percent_values = filtered_main_metric[filtered_main_metric >= threshold]
print("前2%的阈值:", threshold)
print("前2%的值:", top_2_percent_values.values)
print("前2%的值数量:", len(top_2_percent_values))


walker = pyg.walk(df_token_infos_with_metrics)


In [ ]:
df_token_infos_with_metrics

In [ ]:
boxplot_result['fliers'][0].get_data()[1]

In [ ]:
# 获取异常点
outliers = boxplot_result['fliers'][0].get_ydata()
print("异常点:", outliers)

In [ ]:
df_token_infos_with_metrics

In [5]:
from stickytoken.utils import load_vocab_verifications
token_infos_with_metrics_and_part_verifications = load_vocab_verifications(os.path.basename(EXP.MODEL))
import pandas as pd
import pygwalker as pyg
df_token_infos_with_metrics_and_part_verifications = pd.DataFrame(token_infos_with_metrics_and_part_verifications).T
# walker = pyg.walk(df_token_infos_with_metrics_and_part_verifications)

In [6]:
df_token_infos_with_metrics_and_part_verifications['first_verification_element'] = df_token_infos_with_metrics_and_part_verifications['verification'].apply(lambda x: x[0] if isinstance(x, (list, tuple)) else None)


In [ ]:
walker = pyg.walk(df_token_infos_with_metrics_and_part_verifications)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 过滤掉 'main_metric' 列中的 NaN 值
filtered_main_metric = df_token_infos_with_metrics_and_part_verifications['first_verification_element'].dropna()

# 绘制箱线图并获取异常点
plt.figure(figsize=(10, 6))
boxplot_result = plt.boxplot(filtered_main_metric)
plt.title('Boxplot of Main Metric')
plt.ylabel('Main Metric')
plt.show()

# 计算四分位数和四分位间距
Q1 = filtered_main_metric.quantile(0.25)
median = filtered_main_metric.median()
Q3 = filtered_main_metric.quantile(0.75)
IQR = Q3 - Q1

# 计算上下边界
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 找出异常值
outliers = filtered_main_metric[(filtered_main_metric < lower_bound) | (filtered_main_metric > upper_bound)]
print("下四分位数 Q1:", Q1)
print("中位数:", median)
print("上四分位数 Q3:", Q3)
print("四分位间距 IQR:", IQR)
print("下边界:", lower_bound)
print("上边界:", upper_bound)
print("异常值:", outliers.values)
print("异常值数量:", len(outliers))

# 找出filter后的main_metric中的前2%的值
threshold = np.percentile(filtered_main_metric, 98)
top_2_percent_values = filtered_main_metric[filtered_main_metric >= threshold]
print("前2%的阈值:", threshold)
print("前2%的值:", top_2_percent_values.values)
print("前2%的值数量:", len(top_2_percent_values))


# walker = pyg.walk(df_token_infos_with_metrics)


In [ ]:
# from stickytoken.embedding_model import candidates_for_verification
# candidates, threshold = candidates_for_verification(
#                 token_infos_with_metrics_and_part_verifications
#             )
# remaining_candidates = [tc for tc in candidates if "verification" not in tc]

In [ ]:
# moda.adaptive_threshold_verification(verification_dataset,EXP.INSERT_NUM)
# moda.final_verification(token_infos_with_metrics,
#                         verification_dataset,
#                         EXP.INSERT_NUM)
moda.final_verification(token_infos_with_metrics_and_part_verifications,
                        verification_dataset,
                        EXP.INSERT_NUM)

In [16]:
# moda.cosine_threshold,moda.euclidean_threshold,moda.manhattan_threshold

In [ ]:
moda.record_all(EXP)